<a href="https://colab.research.google.com/github/cwinsor/medical_image_uw_madison/blob/main/colab_01_data_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print(torch.__version__);print(torch.version.cuda)

2.0.1+cu118
11.8


In [2]:
import cv2

In [3]:
print(torch.__version__);print(torch.version.cuda)

2.0.1+cu118
11.8
